YoloV3'ü neden Collab üzerinden eğitiriz ? Bu sorunu cevabı Collab'ın Google sunucuları üzerinden biz piyasada bulamayacağız kadar güçlü bir ekran kartı vermesi, indirme işlemlerinin oldukça hızlı olması ve pyhon ile uyumlu - tüm kütüphanelerin kullanılabilir olmasıdır. Başlamadan önce yapmanız gerekn tek şey sunucu ayarlarında işlemci tipini GPU olarak seçmektir. Sonrasında ise bu dosyayı olduğu şekilde çalıştırmanız yeterlidir. Bu kod sadece collab ortamı için hazrılanmıştır bu yüzden kendi bilgisayarınızda çalıştırmayı denemeyin.

Not :: Bu dosya tek objeli modeller eğitmek için hazırlanmıştır.

İlk olarak collab'ın bize atadığı GPU bilgilerine bakmak için . . .

In [ ]:
!nvidia-smi

Daha sonrasında Google Drive'ımız ile bağlantı kurmak için bağlantı izni atıyoruz. Bu hücre çalıştığında karşınıza  onay vermeniz için bir izin ekranı çıkacaktır.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Daha sonra eğitim için Darknet reposunu klonluyoruz. Sonuçta bu hazır bir nöral ağ ve bu yüzden eğitimin de ağ üzerinden gerçekleştirilmesi gerekli.

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

DarkNet'i GPU ile derliyoruz.

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

Buradan sorası default parçalardan oluşuyor. Bizlerin burada yapabileceğim tek değişiklk eğitimi gerçekleştrmek istediğimiz modele göre cfg dosyalarını ayarlayabiliriz. Örneğin bu hücre yolo-tiny'e göre model eğitirken . . .

In [ ]:
!cp cfg/yolov3-tiny.cfg cfg/yolov3_training-tiny.cfg

Bu hücre yolo-320'e göre model eğitimi yapacaktır.

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

Daha sonrasında ise drive'dan eğitim resimlerini içeren zip dosyasını içe aktarıyoruz. İkici hücrede obje isminizde değişiklik yapmayı unutmayın.

In [ ]:
!mkdir "/mydrive/yolov3"

In [ ]:
!echo "Obje İsmi" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

Sonrasında eğitim için gerekli ağırlıkları içe aktarıyoruz.

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

Sonrasında ise içe aktardığımız zip dosyasının içersindeki resimler dışarı çıkarıyoruz ve sırası ile işleme alıyoruz.

In [ ]:
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

Sonrasında train.txt dosyası oluşturyoruz.

In [ ]:
file = open("data/train.txt", "w")
file.write("\n".join(images_list))
file.close()

Artık eğitimi başlatabiliriz. Burada dikkat etmeniz gereken şey average_loss değerinin en düşük haline gelmesidir.

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

# Çok Objeli Modeller İçin
Eğer modeliniz birden fazla obje tanıyor ise cfg (config) dosyasında bazı değişiklikler yapmanız gerekmektedir. Bu değişiklikler cfg dosyası içerisinde [yolo] içerisindeki " classes " ve [convolutional] içerisinde " filters " değerleridir.
Tüm dosyadaki classes değerlerini modelinizin sahip olduğu sınıf sayısında göre ve filters değerlerinide (sınıf sayısı + 5)*3 değerine eşitliyorsunuz. Rktra oalrakta yapabileceğiniz değişiklikler mğmlündğr tabi, max_baches = num of classes * 2000 ,